# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [100]:
import pandas as pd
import cassandra
import os
import glob
import csv

#### Creating list of filepaths to process original event csv data files

In [101]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'
file_path_list=[]
for root, dirs, files in os.walk(filepath):
    files = glob.glob(os.path.join(root,'*'))
    for file in files:
        file_path_list.append(file)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [102]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
# first_file is variable which helps to include header in new csv
first_file=1   
       
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        #to include header of only first file in the new csv
        if first_file==0:
            next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
        first_file=0

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow(row)


In [103]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6932


## Part II. Apache Cassandra portion
#### Processing the newly created event_datafile_new.csv 

#### Creating a Cluster

In [104]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [105]:
try:
    query="create keyspace if not exists udacity\
    with replication = {'class':'SimpleStrategy','replication_factor':1}"
    session.execute(query)
except Exception as e:
    print(e)

#### Set Keyspace

In [106]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries.

### Create queries to ask the following three questions of the data

 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query-1 Model Description

Query1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
  
Approach: 

Here query expects Name of the artist, title of the song and song's length based on sessionId and itemInSession

As we are working here with NoSQL database ,so we decide query first then create table based on that query.

- Expected Output: Name of the artist, title of the song , song's length
- Based On: sessionId , itemInSession

so the required query should be like
```
select artist,song_title,song_length from session_songs where sessionId = value and itemInSession = value

```
now our query is finalised ,we can now move to create table based on above query.

We will add 'NOT EXIST' to the CREATE statement to check if the table exists and only create the table if it does not exist. 
Now we need to select the columns that are going to be in the table and the Primary Key

Primary key: It uniquely identifies a row in a table .
In Cassandra Primary Key is consist of two parts (Partition key,clustering key{optional})

Partition key - It tells how our data is partitioned and stored.
we need to use (sessionId) as our partition key as we are fetching our records based on sessionId.
Clustering key - It tells how our data is stored within the partition.
we need to use itemInSession as our clustering key as there can be multiple rows with same sessionId and we 
want our Primary key to be unique.

so our PRIMARY KEY will be :
```
(sessionId,itemInSession)
```
Now we are ready to go for table creation followed by ETL flow and then execute the query.



#### Create Table

In [107]:
try:
    query="create table if not exists session_songs"
    query=query+"(sessionId int,itemInSession int,artist text,song_title text,\
    song_length float,primary key(sessionId,itemInSession))"
    session.execute(query)
except Exception as e:
    print(e)           

#### ETL 

In [108]:
file='event_datafile_new.csv'
with open(file,mode='r', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header

    for line in csvreader:
        query = "insert into session_songs(sessionId,itemInSession,artist,song_title,song_length)"
        query = query + " values (%s,%s,%s,%s,%s)"       
        try:
            # here in order to fetch required field from the line row ,we use proper index mapping 
            session.execute(query, (int(line[12]),int(line[4]),line[0], line[13],float(line[6])))
        except Exception as e:
            print(line)
            print(e)

#### Fetch the required data

In [109]:
try:
    query="select artist,song_title,song_length from session_songs \
    where sessionId = 338 and itemInSession = 4"
    rows=session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist,row.song_title,row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query-2 Model Description

Query2 : Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
  
Approach: 

Here query expects Name of the artist, title of the song(sorted by itemInSession) and user details (ie.first and last name) based on userId and sessionId.

As we are working here with NoSQL database ,so we decide query first then create table based on that query.

- Expected Output: Name of the artist, title of the song(sorted by itemInSession) , user details(first and last name)
- Based On: userId,sessionId

so the required query should be like
```
select artist,song_title,first_name,last_name from user_events where userId=value and sessionId=value

```
now our query is finalised ,we can now move to create table based on above query.

We will add 'NOT EXIST' to the CREATE statement to check if the table exists and only create the table if it does not exist. 
Now we need to select the columns that are going to be in the table and the Primary Key

Primary key: It uniquely identifies a row in a table .
In Cassandra Primary Key is consist of two parts (Partition key,clustering key{optional})
Partition key - It tells how our data is partitioned and stored
we need to use (userId,sessionId) as our partition key as we are fetching our records based on userId and sessionId.
Clustering key - It tells how our data is stored within the partition.
we need to use itemInSession as our clustering key as we want our data(ie song title) to be sorted by itemInSession.

so our PRIMARY KEY will be :
```
((userId,sessionId),itemInSession)
```
(Partition key,clustering key) as ((userId,sessionId),itemInSession)

Now we are ready to go for table creation followed by ETL flow and then execute the query.

#### Create Table

In [110]:
try:
    query="create table if not exists user_events"
    query=query+"(userId int,sessionId int,itemInSession int,artist text,\
    song_title text,first_name text,last_name text,primary key((userId,sessionId),itemInSession))"
    session.execute(query)
except Exception as e:
    print(e)        

#### ETL

In [111]:
file='event_datafile_new.csv'

with open(file,mode='r',encoding='utf8') as f:
    csv_reader=csv.reader(f)
    next(csv_reader) 
    for line in csv_reader:
        try:
            query="insert into user_events(userId,sessionId,itemInSession,artist,song_title,first_name,last_name) "
            query=query+"values(%s,%s,%s,%s,%s,%s,%s)"
            session.execute(query,(int(line[16]),int(line[12]),int(line[4]),line[0],line[13],line[2],line[5]))
        except Exception as e:
            print(line)
            print(e)

#### Fetch the required data

In [112]:
try:
    query="select artist,song_title,first_name,last_name from user_events\
    where userId=10 and sessionId=182"
    rows=session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist,row.song_title,row.first_name,row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query-3 Model Description

Query3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Approach: 

Here query expects User details(ie.first and last name ) based on the title of song.

As we are working here with NoSQL database ,so we decide query first then create table based on that query.

- Expected Output: FirstName,LastName
- Based On: Title of song

so the required query should be like

```
select first_name,last_name from user_songs_details where song='value'

```
now our query is finalised ,we can now move to create table based on above query.

We will add 'NOT EXIST' to the CREATE statement to check if the table exists and only create the table if it does not exist. 
Now we need to select the columns that are going to be in the table and the Primary Key

Primary key: It uniquely identifies a row in a table .

here Primary Key will be :
```
(song,userId)
```
song as our partition key and userId helps to cluster all userId in that song partition.



Now we are ready to go for table creation followed by ETL flow and then execute the query.


#### Create Table

In [113]:
try:
    query="create table if not exists user_songs_details"
    query=query+"(song text,userId int,first_name text,last_name text,\
    primary key(song,userId))"
    session.execute(query)
except Exception as e:
    print(e)          

#### ETL

In [114]:
file='event_datafile_new.csv'

with open(file,mode='r',encoding='utf8') as f:
    csv_reader=csv.reader(f)
    next(csv_reader)
    for line in csv_reader:
        try:
            query="insert into user_songs_details(song,userId,first_name,last_name) "
            query=query+"values(%s,%s,%s,%s)"
            session.execute(query,(line[13],int(line[16]),line[2],line[5]))
        except Exception as e:
            print(line)
            print(e)

#### Fetch the required data

In [115]:
try:
    query="select first_name,last_name from user_songs_details\
    where song='All Hands Against His Own'"
    rows=session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.first_name,row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [116]:
try:
    session.execute("drop table if exists session_songs")
    session.execute("drop table if exists user_events")
    session.execute("drop table if exists user_songs_details")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [117]:

session.shutdown()
cluster.shutdown()